In [1]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("2.1.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


In [2]:
!pip install matplotlib==3.5.2
!pip install git+https://github.com/openai/CLIP.git
#!pip install torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1

!pip install kaolin==0.15.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.1.0_cu121.html
!pip install open3d

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_a2b2ca1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_a2b2ca1
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
Looking in links: https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.1.0_cu121.html


In [3]:

import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from math import radians
from pytorch3d.transforms import axis_angle_to_matrix
# Util function for loading point clouds|
import numpy as np

# Data structures and functions for rendering
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.renderer import (
    look_at_rotation,
    look_at_view_transform,
    FoVOrthographicCameras,
    PointsRasterizationSettings,
    PointsRenderer,
    PulsarPointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    NormWeightedCompositor,
    FoVPerspectiveCameras,
    PerspectiveCameras,
    rotate_on_spot,
    OrthographicCameras
)

In [95]:
def render_point_cloud(point_cloud,distance,elev,azim,a1,a2,a3):

  R, T = look_at_view_transform(distance, elev, azim)

  angles = [radians(a1), radians(a2),radians(a3)]
  rotation = axis_angle_to_matrix(torch.FloatTensor(angles))
  R, T = rotate_on_spot(R, T, rotation)
  cameras = FoVPerspectiveCameras(device=device, R=R, T=T, znear=0.01)#questa la fa di fronte

  # Define the settings for rasterization and shading. Here we set the output image to be of size
  # 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
  # and blur_radius=0.0. Refer to raster_points.py for explanations of these parameters.
  raster_settings = PointsRasterizationSettings(
      image_size=512,
      radius = 0.003,
      points_per_pixel = 10
  )
  renderer = PointsRenderer(
      rasterizer=PointsRasterizer(cameras=cameras, raster_settings=raster_settings),
      compositor=NormWeightedCompositor()
  )


  return renderer(point_cloud)
  #plt.figure(figsize=(10, 10))
  #plt.imshow(images[0, ..., :3].cpu().numpy())
  #plt.axis("off");
def resize(image,res):
  resized=image.permute(3, 1, 2, 0).squeeze(3)

  transform = transforms.Resize((res, res))
  out=transform(resized)
  return out


def generate_static_views(point_cloud,res):


  mylist = [ resize(render_point_cloud(point_cloud,3,0,0,0,0,0),res),
             resize(render_point_cloud(point_cloud,3,0,0,0,0,540),res),
             resize(render_point_cloud(point_cloud,3,90,0,0,0,0),res),
             resize(render_point_cloud(point_cloud,3,-80,-2,0,0,0),res),
             resize(render_point_cloud(point_cloud,3,0,50,0,0,0),res)
  ]

  return mylist






In [6]:
# Setup

import open3d as o3d
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")


def segment2rgb(pred_class, colors):
    pred_rgb = torch.zeros(pred_class.shape[0], 3).to(device)
    for class_idx, color in enumerate(colors):
        pred_rgb += torch.matmul(pred_class[:,class_idx].unsqueeze(1), color.unsqueeze(0))

    return pred_rgb

def color_point_cloud(pred_class, verts, colors):
    pred_rgb = segment2rgb(pred_class, colors)
    #print(pred_rgb.shape)
    #print(verts.shape[0])

    return Pointclouds(points=[verts], features=[pred_rgb])




In [7]:
#!pip install --upgrade pip
!git clone https://github.com/rasenqt/AML_Project_2024.git
#!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2


fatal: destination path 'AML_Project_2024' already exists and is not an empty directory.


In [8]:
import argparse
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torchvision
import sys
import os
import argparse
sys.path.append('/content/AML_Project_2024/3DHighlighter-main')
from itertools import permutations, product
from neural_highlighter import NeuralHighlighter
from Normalization import MeshNormalizer,PointCloudNormalizer
from mesh import Mesh
from pathlib import Path
from render import Renderer
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from utils import device, color_mesh


def optimize_point_cloud(args):
    # Constrain most sources of randomness
    # (some torch backwards functions within CLIP are non-determinstic)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    # Load CLIP model
    clip_model, preprocess = clip.load(args.clipmodel, device, jit=args.jit)

    # Adjust output resolution depending on model type
    res = 224
    if args.clipmodel == "ViT-L/14@336px":
        res = 336
    if args.clipmodel == "RN50x4":
        res = 288
    if args.clipmodel == "RN50x16":
        res = 384
    if args.clipmodel == "RN50x64":
        res = 448

    Path(os.path.join(args.output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

    objbase, extension = os.path.splitext(os.path.basename(args.obj_path))
    pointcloud = o3d.io.read_point_cloud(args.obj_path)
    verts = torch.Tensor(np.asarray(pointcloud.points)).to(device)
    print(verts.shape)


    # Manually set RGB color (assuming you want to set all points to red)
    manual_color = torch.tensor([0.0, 1.0, 0.0], dtype=torch.float32, device=device)

    # Repeat the manual color for each vertex
    rgb_colors = manual_color.repeat(verts.shape[0], 1)

    shift = torch.mean(verts, dim=0)
    scale = torch.max(torch.norm(verts-shift, p=2, dim=1))
    verts=(verts-shift)/scale

    point_cloud = Pointclouds(points=[verts], features=[rgb_colors])
    #PointCloudNormalizer(point_cloud)()




    # Initialize variables
    background = None
    if args.background is not None:
        assert len(args.background) == 3
        background = torch.tensor(args.background).to(device)
    n_augs = args.n_augs
    dir = args.output_dir

    # Record command line arguments
    with open(os.path.join(dir, 'commandline_args.txt'), 'w') as f:
        json.dump(args.__dict__, f, indent=2)

    # CLIP and Augmentation Transforms
    clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
    clip_transform = transforms.Compose([
        transforms.Resize((res, res)),
        clip_normalizer
    ])
    augment_transform = transforms.Compose([
        transforms.RandomResizedCrop(res, scale=(1, 1)),
        transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
        clip_normalizer
    ])

    # MLP Settings
    mlp = NeuralHighlighter(args.depth, args.width, out_dim=args.n_classes, positional_encoding=args.positional_encoding,
                            sigma=args.sigma).to(device)
    optim = torch.optim.Adam(mlp.parameters(), args.learning_rate)

    # list of possible colors
    rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
    color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
    full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
    colors = torch.tensor(full_colors).to(device)


    # --- Prompt ---
    # pre-process multi_word_inputs
    args.object[0] = ' '.join(args.object[0].split('_'))
    for i in range(len(args.classes)):
        args.classes[i] = ' '.join(args.classes[i].split('_'))
    # encode prompt with CLIP
    prompt = "A 3D render of a gray {} with highlighted {}".format(args.object[0], args.classes[0])
    with torch.no_grad():
        prompt_token = clip.tokenize([prompt]).to(device)
        encoded_text = clip_model.encode_text(prompt_token)
        encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)

    vertices = copy.deepcopy(verts)
    losses = []

    # Optimization loop
    for i in tqdm(range(args.n_iter)):
        optim.zero_grad()

        # predict highlight probabilities
        pred_class = mlp(vertices)

        # color and render mesh
        verts_curr = vertices
        colored_point_cloud = color_point_cloud(pred_class, verts_curr, colors)
        rendered_images = generate_static_views(colored_point_cloud,res)

        # Calculate CLIP Loss
        loss = clip_loss(args, rendered_images, encoded_text, clip_transform, augment_transform, clip_model)
        loss.backward(retain_graph=True)

        optim.step()

        # update variables + record loss
        with torch.no_grad():
            losses.append(loss.item())

        # report results
        if i % 100 == 0:
            print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
            save_renders(dir, i, rendered_images)
            with open(os.path.join(dir, "training_info.txt"), "a") as f:
                f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")

    # re-initialize background color
    if args.background is not None:
        assert len(args.background) == 3
        background = torch.tensor(args.background).to(device)
    # save results
    #save_final_results(args, dir, vertices, mlp, vertices, colors, render, background)

    # Save prompts
    with open(os.path.join(dir, prompt), "w") as f:
        f.write('')



# ================== HELPER FUNCTIONS =============================
def save_final_results(args, dir, mesh, mlp, vertices, colors, render, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)
        # for renders
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)
        sampled_mesh = mesh

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight/255, gray/255)).to(device)
        color_mesh(one_hot, sampled_mesh, colors)
        rendered_images, _, _ = render.render_views(sampled_mesh, num_views=args.n_views,
                                                                        show=args.show,
                                                                        center_azim=args.frontview_center[0],
                                                                        center_elev=args.frontview_center[1],
                                                                        std=args.frontview_std,
                                                                        return_views=True,
                                                                        lighting=True,
                                                                        background=background)
        # for mesh
        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx==0, highlight, gray)
        objbase, extension = os.path.splitext(os.path.basename(args.obj_path))
        mesh.export(os.path.join(dir, f"{objbase}_{args.classes[0]}.ply"), extension="ply", color=final_color)
        save_renders(dir, 0, rendered_images, name='final_render.jpg')


def clip_loss(args, rendered_images, encoded_text, clip_transform, augment_transform, clip_model):
    if args.n_augs == 0:
        clip_image = clip_transform(rendered_images)
        encoded_renders = clip_model.encode_image(clip_image)
        encoded_renders = encoded_renders / encoded_renders.norm(dim=1, keepdim=True)
        if args.clipavg == "view":
            if encoded_text.shape[0] > 1:
                loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                torch.mean(encoded_text, dim=0), dim=0)
            else:
                loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                encoded_text)
        else:
            loss = torch.mean(torch.cosine_similarity(encoded_renders, encoded_text))
    elif args.n_augs > 0:
        loss = 0.0
        for _ in range(args.n_augs):
            augmented_image = augment_transform(rendered_images)
            encoded_renders = clip_model.encode_image(augmented_image)
            if args.clipavg == "view":
                if encoded_text.shape[0] > 1:
                    loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                    torch.mean(encoded_text, dim=0), dim=0)
                else:
                    loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                    encoded_text)
            else:
                loss -= torch.mean(torch.cosine_similarity(encoded_renders, encoded_text))
    return loss

def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))



In [9]:
import sys
import os
from io import StringIO
import argparse
sys.path.append('/content/AML_Project_2024/3DHighlighter-main/')
from main import optimize # Import the function from the module

class ArgsNamespace:
    def __init__(self, args_dict):
        self.__dict__.update(args_dict)

# Create an instance of ArgsNamespace



# specify the directory you want to iterate over
#directory = './AML_Project_2024/point_cloud_dataset/'  # current directory
directory= './AML_Project_2024/dataset/'

# iterate over files in the directory

for filename in os.listdir(directory):
    #if(filename.split('.')[1]=='obj'):
     # continue

    object_name = filename.split('.')[0]
    print(object_name)


    args = ArgsNamespace({
        'seed': 0,
        'obj_path': directory + filename,
        'output_dir': '/content/results_' + object_name,
        'object': [object_name],
        'classes': ['shoes'],
        'background': [1., 1., 1.],
        'n_views': 5,
        'frontview_std': 4,
        'frontview_center': [0., 0.],
        'show': False,
        'n_augs': 1,
        'clipavg': 'view',
        'render_res': 224,
        'clipmodel': 'ViT-L/14',
        'jit': False,
        'depth': 4,
        'width': 256,
        'n_classes': 2,
        'positional_encoding': False,
        'sigma': 5.0,
        'learning_rate': 0.0001,
        'n_iter': 1
    })
    # Redirect standard output to capture print statements
    old_stdout = sys.stdout
    new_stdout = StringIO()
    sys.stdout = new_stdout

    try:
        optimize(args)
    finally:
        # Reset the standard output
        sys.stdout = old_stdout

    # Print captured output
    print("Output from optimize function:")
    print(new_stdout.getvalue())
    print("porco dio")
    break  # You may want to remove this break depending on your requirements


"""

from google.colab import files

!zip -r /content/results_television.zip /content/results_television
files.download('/content/results_television.zip')

"""

bowtie


100%|██████████| 1/1 [00:00<00:00,  1.65it/s]


Output from optimize function:
ModuleList(
  (0): Linear(in_features=3, out_features=256, bias=True)
  (1): ReLU()
  (2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): ReLU()
  (8): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (9): Linear(in_features=256, out_features=256, bias=True)
  (10): ReLU()
  (11): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (12): Linear(in_features=256, out_features=256, bias=True)
  (13): ReLU()
  (14): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (15): Linear(in_features=256, out_features=2, bias=True)
  (16): Softmax(dim=1)
)
<built-in method type of Tensor object at 0x7f1b701da2f0>
torch.Size([3, 224, 224])
Last 100 CLIP score: -0.2734375
DIOCAROGNAAa

porco dio


"\n\nfrom google.colab import files\n\n!zip -r /content/results_television.zip /content/results_television\nfiles.download('/content/results_television.zip')\n\n"